In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

In [10]:
# Historical fixtures played by players
lineup_df = pd.read_csv('../../data/fbref/games_lineup.csv')
lineup_df = lineup_df.loc[lineup_df.Player == 'Chris Wood']
lineup_df['team'] = np.where(lineup_df.home == 1, lineup_df.squad_h, lineup_df.squad_a)
lineup_df["date"] = pd.to_datetime(lineup_df["date"])
lineup_df = lineup_df.sort_values(by=['date'], ascending=True).reset_index(drop=True)
lineup_df.head()

,Player,Lineup,Starter,Benched,home,Min,date,squad_h,squad_a,team
0,Chris Wood,1,0.0,1.0,0,34.0,2017-08-27,Tottenham,Burnley,Burnley
1,Chris Wood,1,1.0,0.0,1,90.0,2017-09-10,Burnley,Crystal Palace,Burnley
2,Chris Wood,1,1.0,0.0,0,86.0,2017-09-16,Liverpool,Burnley,Burnley
3,Chris Wood,1,1.0,0.0,1,90.0,2017-09-23,Burnley,Huddersfield,Burnley
4,Chris Wood,1,1.0,0.0,0,90.0,2017-10-01,Everton,Burnley,Burnley


In [14]:
lineup_df.shape

(157, 10)

In [11]:
# Historical PL Fixtures
fixtures_df = pd.read_csv("../../data/fbref/fixtures.csv")
fixtures_df = fixtures_df.loc[fixtures_df.Competition == 'Premier-League']
fixtures_df = fixtures_df[["Date", "Home", "Away"]]
fixtures_df["Date"] = pd.to_datetime(fixtures_df["Date"])
fixtures_df = fixtures_df.sort_values(by=['Date'], ascending=True).reset_index(drop=True)
fixtures_df.head()

,Date,Home,Away
0,2017-08-11,Arsenal,Leicester City
1,2017-08-12,Brighton,Manchester City
2,2017-08-12,Watford,Liverpool
3,2017-08-12,Crystal Palace,Huddersfield
4,2017-08-12,West Brom,Bournemouth


In [26]:
# Add data when the player is not included in the team
# And keep only fixtures of the current team
all_fixtures = pd.merge(
    lineup_df,
    fixtures_df,
    left_on=['date', 'team'],
    right_on=['Date', 'Home'],
    how='right'
)
all_fixtures

,Player,Lineup,Starter,Benched,home,Min,date,squad_h,squad_a,team,Date,Home,Away
0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2017-08-11,Arsenal,Leicester City
1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2017-08-12,Brighton,Manchester City
2,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2017-08-12,Watford,Liverpool
3,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2017-08-12,Crystal Palace,Huddersfield
4,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2017-08-12,West Brom,Bournemouth
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2022-05-22,Leicester City,Southampton
1896,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2022-05-22,Brentford,Leeds United
1897,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2022-05-22,Burnley,Newcastle Utd
1898,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2022-05-22,Crystal Palace,Manchester Utd


In [59]:
# Get players listed in current EPL season
players_df = (
    pd.read_csv('../../data/fpl_official/vaastav/data/2021-22/players_raw.csv')
    [['first_name', 'second_name', 'web_name', 'team']])

# Get possible names (complicated because of Brazilian & Portugese naming conventions) 
players_df['full_name'] = players_df['first_name'] + ' ' + players_df['second_name']

players_df = pd.merge(
    players_df,
    pd.read_csv('../../data/fpl_official/vaastav/data/2021-22/id_dict.csv')[[' Understat_Name', ' FPL_Name']],
    left_on='full_name',
    right_on=' Understat_Name',
    how='outer'
)

players_df = pd.merge(
    players_df,
    pd.read_csv('../../data/fpl_official/vaastav/data/2021-22/id_dict.csv')[[' Understat_Name', ' FPL_Name']],
    left_on='full_name',
    right_on=' FPL_Name',
    how='outer'
)

# Add their team
players_df = pd.merge(
    players_df,
    pd.read_csv('../../data/fpl_official/vaastav/data/2021-22/teams.csv')[['id', 'name']],
    left_on='team',
    right_on='id',
).drop('id', axis=1)

#
def foo(x):
    if not pd.isna(x[' Understat_Name_x']):
        return x[' Understat_Name_x']

    if not pd.isna(x[' Understat_Name_y']):
        return x[' Understat_Name_y']
    
    return None

players_df['name'] = players_df.apply(lambda x: foo(x), axis=1)
players_df = players_df[['team', 'full_name', 'name']]
# players_df.loc[players_df.team == 1]

In [8]:
def player_match_history(name, club):
    # Player minutes
    lineup_df = pd.read_csv('../../data/fbref/games_lineup.csv')
    lineup_df['team'] = np.where(lineup_df.home == 1, lineup_df.squad_h, lineup_df.squad_a)
    lineup_df = lineup_df.loc[lineup_df.Player == name].loc[lineup_df.team == club]
    lineup_df["date"] = pd.to_datetime(lineup_df["date"])

    # Historical PL Fixtures
    fixtures_df = pd.read_csv("../../data/fbref/fixtures.csv")
    fixtures_df = fixtures_df.loc[fixtures_df.Competition == 'Premier-League']
    fixtures_df = fixtures_df[["Date", "Home", "Away"]]
    fixtures_df["Date"] = pd.to_datetime(fixtures_df["Date"])

    # Add data when the player is not included in the team
    # And keep only fixtures of the current team
    all_fixtures = fixtures_df.loc[(fixtures_df.Home == club) | (fixtures_df.Away == club)]
    all_fixtures = pd.merge(
        lineup_df,
        all_fixtures,
        left_on=['date', 'squad_h', 'squad_a'],
        right_on=['Date', 'Home', 'Away'],
        how='right'
    )
    all_fixtures.Player = name
    all_fixtures = all_fixtures.fillna(0)
    
    # Remove fixtures before the player is ever in the team (transfer or too young)
    all_fixtures = all_fixtures.sort_values(by=['Date'], ascending=True).reset_index(drop=True)
    all_fixtures = all_fixtures.loc[np.cumsum(all_fixtures['Lineup']) >= 1]

    # Complete data
    all_fixtures.home = np.where(all_fixtures.Home == club, 1, 0)
    all_fixtures = all_fixtures.drop(['date', 'squad_h', 'squad_a'], axis=1)
    all_fixtures['Subbed on'] = np.where((all_fixtures.Benched == 1) & (all_fixtures.Min > 0), 1, 0)
    all_fixtures['Subbed off'] = np.where((all_fixtures.Starter == 1) & (all_fixtures.Min < 90), 1, 0)

    all_fixtures['A'] = np.where(all_fixtures.Min == 90, 1, 0)
    all_fixtures['B'] = np.where(all_fixtures['Subbed off'] == 1, 1, 0)
    all_fixtures['C'] = np.where(all_fixtures.Min == 0, 1, 0)
    all_fixtures['D'] = np.where(all_fixtures['Subbed on'] == 1, 1, 0)

    # Keep fixtures that were played
    return all_fixtures.loc[all_fixtures.Date < datetime.today()]

In [29]:
# player_match_history('Diogo Jota', 'Wolves')
lineup_df = pd.read_csv('../../data/fbref/games_lineup.csv')
# lineup_df = lineup_df.loc[lineup_df.Player == 'Diogo Jota'].loc[(lineup_df.squad_h == 'Wolves') | (lineup_df.squad_a == 'Wolves')]
# lineup_df["date"] = pd.to_datetime(lineup_df["date"])
lineup_df

,Player,Lineup,Starter,Benched,home,Min,date,squad_h,squad_a
0,David Raya,1,1.0,0.0,1,90.0,2021-08-13,Brentford,Arsenal
1,Rico Henry,1,1.0,0.0,1,90.0,2021-08-13,Brentford,Arsenal
2,Ethan Pinnock,1,1.0,0.0,1,90.0,2021-08-13,Brentford,Arsenal
3,Christian Nørgaard,1,1.0,0.0,1,90.0,2021-08-13,Brentford,Arsenal
4,Sergi Canós,1,1.0,0.0,1,90.0,2021-08-13,Brentford,Arsenal
...,...,...,...,...,...,...,...,...,...
68354,Harvey Barnes,1,0.0,1.0,0,6.0,2018-05-13,Tottenham,Leicester City
68355,Fousseni Diabaté,1,0.0,1.0,0,30.0,2018-05-13,Tottenham,Leicester City
68356,Layton Ndukwu,1,0.0,1.0,0,0.0,2018-05-13,Tottenham,Leicester City
68357,Hamza Choudhury,1,0.0,1.0,0,35.0,2018-05-13,Tottenham,Leicester City


In [9]:
from tqdm import tqdm
df = []

for player in tqdm(players_df.iterrows()):
    player_matches = player_match_history(
            player[1]['first_name'] + ' ' + player[1]['second_name'],
            player[1]['name']
        )

    if player_matches.shape[0]:
        df.append(player_matches)

df = pd.concat(df).reset_index(drop=True)

722it [01:24,  8.59it/s]


In [10]:
# NOTE: Formula for when computing features for all the players in the same DF
start = 1
end = 4
pd.concat(
    [
        player_match_history('Mohamed Salah', 'Liverpool'),
        player_match_history('Matt Doherty', 'Tottenham')]
    ).groupby(['Player'])['Min'].apply(lambda group: group.shift(start).rolling(end - start + 1).mean())

0        NaN
1        NaN
2        NaN
3        NaN
4      62.50
       ...  
181    66.75
182    87.25
182    61.25
183    69.75
183    64.25
Name: Min, Length: 320, dtype: float64

In [57]:
# Days since last game
df = pd.read_csv("../../data/fbref/fixtures.csv")
df = df.loc[df.Competition == 'Premier-League']
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Date'].dropna(inplace=True)

df = pd.concat([
    df[['Date', 'Home']].rename(columns={'Home': 'Team'}),
    df[['Date', 'Away']].rename(columns={'Away': 'Team'})
])
df = df.sort_values(by=['Date'], ascending=True).reset_index(drop=True)

df['delta'] = df.groupby('Team')['Date'].apply(lambda group: group.diff())

In [66]:
# Importance
# TODO: Change names to match between fbref & 538
pd.merge(
    pd.read_csv("../../data/fbref/fixtures.csv"),
    pd.read_csv("../../data/fivethirtyeight/spi_matches.csv")[['date', 'team1', 'importance1', 'importance2', 'spi1', 'spi2']],
    left_on=['Date', 'Home'],
    right_on=['date', 'team1'],
    )

,Round,Day,Date,Time,Home,Score,Away,Attendance,Venue,Referee,Notes,Competition,date,team1,importance1,importance2,spi1,spi2
0,1.0,Fri,2021-08-13,20:00,Brentford,2–0,Arsenal,16479.0,Brentford Community Stadium,Michael Oliver,NaN,Premier-League,2021-08-13,Brentford,39.2,44.5,62.47,80.58
1,1.0,Sat,2021-08-14,15:00,Leicester City,1–0,Wolves,31983.0,King Power Stadium,Craig Pawson,NaN,Premier-League,2021-08-14,Leicester City,39.1,21.5,77.57,70.14
2,1.0,Sat,2021-08-14,15:00,Burnley,1–2,Brighton,16910.0,Turf Moor,David Coote,NaN,Premier-League,2021-08-14,Burnley,34.4,23.0,64.53,73.89
3,1.0,Sat,2021-08-14,15:00,Chelsea,3–0,Crystal Palace,38965.0,Stamford Bridge,Jonathan Moss,NaN,Premier-League,2021-08-14,Chelsea,57.3,37.4,88.33,62.22
4,1.0,Sat,2021-08-14,15:00,Watford,3–2,Aston Villa,20051.0,Vicarage Road Stadium,Mike Dean,NaN,Premier-League,2021-08-14,Watford,41.1,21.7,60.59,72.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,38.0,Sun,2018-05-13,15:00,Swansea City,1–2,Stoke City,20673.0,Liberty Stadium,Anthony Taylor,NaN,Premier-League,2018-05-13,Swansea City,0.3,0.0,49.23,52.03
1178,38.0,Sun,2018-05-13,15:00,Liverpool,4–0,Brighton,50752.0,Anfield,Kevin Friend,NaN,Premier-League,2018-05-13,Liverpool,100.0,0.0,87.19,56.04
1179,38.0,Sun,2018-05-13,15:00,Burnley,1–2,Bournemouth,20720.0,Turf Moor,Paul Tierney,NaN,Premier-League,2018-05-13,Burnley,0.0,0.0,59.71,56.15
1180,38.0,Sun,2018-05-13,15:00,Crystal Palace,2–0,West Brom,25357.0,Selhurst Park,Jonathan Moss,NaN,Premier-League,2018-05-13,Crystal Palace,0.0,0.0,65.97,56.01
